# Setup

In [ ]:
!pip install openai==1.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
import openai
import logging
from openai import OpenAI
import copy

print(openai.__version__) # version - 1.12.0


1.12.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
cd /content/LLM_MVP


/content/LLM_MVP


In [ ]:
pwd

'/content/LLM_MVP'

### storing real credentials.json inside a google drive folder is not secure and not best practice
### please download the folder, create a github repo, then create a .gitignore

In [ ]:
ls

credentials.json  data_2.json  data_3.json  data.json  starter_code.ipynb


# Load data and credentials

In [ ]:
import json

# Update the path if your file is in a different location
file_path = 'data.json'
file_path_2 = 'data_2.json'
file_path_3 = 'data_3.json'

with open(file_path, 'r') as file:
    patient_data = json.load(file)
with open(file_path_2, 'r') as file:
    patient_data_2 = json.load(file)
with open(file_path_3, 'r') as file:
    patient_data_3 = json.load(file)

# To verify, print a part of the data
print(patient_data['patient_demographics'])
print(patient_data_2['patient_demographics'])
print(patient_data_3['patient_demographics'])

{'name': 'John Doe', 'age': 70, 'gender': 'Male', 'admission_date': '2024-02-10', 'discharge_date': '2024-02-14'}
{'name': 'Michael Thompson', 'age': 58, 'gender': 'Male', 'admission_date': '2024-03-01', 'discharge_date': '2024-03-06'}
{'name': 'Jane Doe', 'age': 65, 'gender': 'Female', 'admission_date': '2024-03-10', 'expected_discharge_date': '2024-03-20'}


In [ ]:
patient_base = {}
def add_patient(patient_data):
  if patient_data['patient_id'] not in patient_base:
    patient_base[patient_data['patient_id']] = patient_data
    print(f"Add {patient_data['patient_id']} into the patient database.")
    return "Added Successfully!"
  else:
    return "Patient ID is already esist! Pelease double check your data!"


In [ ]:
print("a\nb")

a
b


In [ ]:
def privacy_handler(patient_data):
  patient = copy.deepcopy(patient_data)
  if 'name' in patient['patient_demographics']:
    p_name =  patient['patient_demographics']['name']
    del patient['patient_demographics']['name']
  else: p_name = None
  if 'patient_id' in patient:
    p_id = patient['patient_id']
    del patient['patient_id']
  else: p_id = None
  return p_id, p_name, patient

In [ ]:
a,b,c = privacy_handler(patient_data)
b

'John Doe'

In [ ]:
patient_data

{'patient_id': '123456',
 'patient_demographics': {'name': 'John Doe',
  'age': 70,
  'gender': 'Male',
  'admission_date': '2024-02-10',
  'discharge_date': '2024-02-14'},
 'diagnoses': [{'date': '2024-02-10',
   'diagnosis_code': 'J18.1',
   'description': 'Lobar pneumonia, unspecified organism'}],
 'drg': {'code': '193',
  'description': 'Simple pneumonia and pleurisy with MCC'},
 'encounters': [{'date': '2024-02-10',
   'type': 'Admission',
   'reason': 'Cough, shortness of breath, hemoptysis, fever'},
  {'date': '2024-02-14', 'type': 'Discharge'}],
 'flowsheets': [{'date': '2024-02-10',
   'time': '08:00',
   'temperature': '38.5°C',
   'heart_rate': '90 bpm',
   'blood_pressure': '130/85 mmHg',
   'respiratory_rate': '20 breaths/min',
   'oxygen_saturation': '92%'},
  {'date': '2024-02-11',
   'time': '08:00',
   'temperature': '37.8°C',
   'heart_rate': '85 bpm',
   'blood_pressure': '125/80 mmHg',
   'respiratory_rate': '18 breaths/min',
   'oxygen_saturation': '94%'},
  {'date

In [ ]:
add_patient(patient_data)


'Patient ID is already esist! Pelease double check your data!'

In [ ]:
'123456' in patient_base

True

In [ ]:
patient_data.keys()

dict_keys(['patient_id', 'patient_demographics', 'diagnoses', 'drg', 'encounters', 'flowsheets', 'imaging', 'labs', 'med_orders', 'notes'])

In [ ]:
# You should not store API keys in a google drive folder - it is not secure. If using github, add credentials.json to .gitignore.

credentials_path = 'credentials.json'

with open(credentials_path, 'r') as file:
    credentials = json.load(file)

openai_api_key = credentials['openai_api_key']

# Use the API key as needed, for example, to configure OpenAI's API client
openai.api_key = openai_api_key


# Explore data

In [ ]:
print(f'Data type: {type(patient_data)}')

Data type: <class 'dict'>


In [ ]:
print(f'Keys in data: {(patient_data_3.keys())}')

Keys in data: dict_keys(['patient_id', 'patient_demographics', 'diagnoses', 'encounters', 'labs', 'med_orders', 'ward_round_notes'])


In [ ]:
# Look at encounters data, there are multiple
patient_data_3['encounters']

[{'date': '2024-03-10',
  'type': 'Admission',
  'reason': 'Fall and subsequent loss of consciousness'},
 {'date': '2024-03-11',
  'type': 'CT Scan',
  'findings': 'Hemorrhagic stroke identified in the right hemisphere'}]

In [ ]:
# Look at flowsheets data, there are multiple
patient_data_3['ward_round_notes']

[{'date': '2024-03-10',
  'time': 'Evening',
  'note': 'Patient experienced a fall last night, now with decreased level of consciousness. GCS recorded at 11. Slurring of speech noted.'},
 {'date': '2024-03-11',
  'time': 'Morning',
  'note': 'CT scan confirms hemorrhagic stroke. Starting mannitol for cerebral edema. Neurological examination: right-sided weakness, continued slurred speech, GCS 11. Patient not safe for discharge, requires close monitoring.'},
 {'date': '2024-03-11',
  'time': 'Evening',
  'note': 'No significant improvement in neurological status. GCS remains at 11. Patient shows signs of agitation and confusion. Safety measures in place to prevent further falls.'},
 {'date': '2024-03-12',
  'time': 'Morning',
  'note': "Patient's condition remains critical. Persistent slurred speech and right-sided weakness. Discussion with family about prognosis and ongoing care needs. Patient not safe for discharge."}]

In [ ]:
patient_data_3['labs']

[{'date': '2024-03-11',
  'tests': [{'name': 'CBC', 'result': 'Hemoglobin slightly low, WBC elevated'},
   {'name': 'Coagulation profile',
    'result': 'INR elevated, suggesting prolonged bleeding time'}]}]

# Sample zero-shot solution
This is a first attempt where we pass in everything and provide basic prompt engineering. This is a baseline solution. Your solutions should improve from this.

In [ ]:
client = OpenAI(api_key=openai_api_key)

In [ ]:
def safety_check(patient_data, additional_prompts=''):
    """
    Generates safety check result using OpenAI's GPT-3.5.

    Parameters:
    - patient_data (str): The patient data to generate the summary from.
    - additional_prompts (str): Try prompt engineering here.

    Returns:
    - str: The generated patient summary.
    """

    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"""
                              You are a doctor writing a discharge letter for a patient. Use patient data from context only. Decide if the patient is feasible to discharge or not.
                              {additional_prompts}
                              Data: {patient_data}
                Output should strictly be a boolean variable: True/False

                """,
            }
        ],
        #model="gpt-4-turbo-preview",
        model="gpt-3.5-turbo",
    )

    # Return the content of the generated message
    return chat_completion.choices[0].message.content

In [ ]:
safety_check(patient_data_3)

'False'

In [ ]:
# Set up basic configuration for logging
log_filename = '/content/LLM_MVP/app_logs.log'
logging.basicConfig(filename=log_filename, filemode='a', format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO)

In [ ]:
def generate_patient_summary(patient_data, additional_prompts=''):
    """
    Generates a patient summary using OpenAI's GPT-3.5.

    Parameters:
    - patient_data (str): The patient data to generate the summary from.
    - additional_prompts (str): Try prompt engineering here.

    Returns:
    - str: The generated patient summary.
    """
    try:
      # Initialize the OpenAI client with your API key
      client = OpenAI(api_key=openai_api_key)

      # Create the chat completion request with the patient data
      chat_completion = client.chat.completions.create(
          messages=[
              {
                  "role": "user",
                  "content": f"""
                                You are a doctor writing a discharge letter for a patient. Use patient data from context only. If this patient is not safe for discharge, do not recommend discharge. Minimize hallucinations.
                                {additional_prompts}
                                Data: {patient_data}

                  """,
              }
          ],
          #model="gpt-4-turbo-preview",
          model="gpt-3.5-turbo",
      )

      # Return the content of the generated message
      # Log successful generation
      logging.info("Successfully generated patient summary.")
      return chat_completion.choices[0].message.content
    except Exception as e:
        # Log the exception
      logging.error("Error in generating patient summary", exc_info=True)
      return f"An error occurred: {str(e)}"

In [ ]:
!tail -f app_logs.log

tail: cannot open 'app_logs.log' for reading: No such file or directory
tail: no files remaining


In [ ]:
print(generate_patient_summary(patient_data))

Discharge Letter:

Patient: John Doe
Patient ID: 123456
Age: 70
Gender: Male

Diagnosis: Lobar pneumonia, unspecified organism (J18.1)
DRG Code: 193 - Simple pneumonia and pleurisy with MCC

Admission Date: February 10, 2024
Discharge Date: February 14, 2024

Dear Mr. Doe,

I am writing to inform you that based on your recent progress and response to treatment for lobar pneumonia, you are now medically fit for discharge. Your condition has significantly improved with the prescribed antibiotics, and your vital signs have stabilized within normal ranges. Your temperature has normalized, and oxygen saturation levels have been consistent.

You are being discharged with instructions to continue taking oral antibiotics for the next 5 days. It is crucial to complete the full course of antibiotics as prescribed to ensure a complete recovery and prevent any relapse. Additionally, you are scheduled for a follow-up appointment at the outpatient clinic in two weeks to monitor your progress and ens

In [ ]:
print(generate_patient_summary(patient_data_2))

**Patient Discharge Letter**

**Patient ID:** 789123  
**Patient Name:** Michael Thompson  
**Age/Gender:** 58, Male  
**Admission Date:** 2024-03-01  
**Discharge Date:** 2024-03-06  

**Admission Diagnoses:**  
- Acute transmural myocardial infarction of anterior wall (I21.0) on 2024-03-01.

**Summary of Hospital Course:**  
Mr. Thompson was admitted on March 1, 2024, with severe chest pain radiating to the left arm and shortness of breath. An ECG confirmed a diagnosis of ST-elevation myocardial infarction (STEMI). He underwent successful Percutaneous Coronary Intervention (PCI) with stent placement on the same day.  
Post-procedure, the patient remained stable with no recurrent chest pain. Troponin levels were elevated upon admission but began to decrease by March 2, 2024, indicating a positive response to the treatment. His condition continued to improve, with stable vital signs and a decreasing trend in troponin levels, supporting his recovery process.  
Throughout his hospital st

In [ ]:
print(generate_patient_summary(patient_data_3))

Discharge Letter:

Patient Name: Jane Doe
Patient ID: 456789
Age: 65
Gender: Female

Admission Date: March 10, 2024
Expected Discharge Date: March 20, 2024

Diagnosis:
- Intracerebral hemorrhage, unspecified (I61.9)
- Hemorrhagic stroke identified in the right hemisphere

Summary:
Jane Doe was admitted on March 10, 2024, due to a fall resulting in a hemorrhagic stroke. She has exhibited symptoms of slurred speech, right-sided weakness, and decreased consciousness since admission. Despite medical intervention and close monitoring, there has been no significant improvement in her neurological status. Additionally, she has shown signs of agitation and confusion, which indicate a need for continued care and observation.

Recommendation:
Due to her critical condition and ongoing symptoms, it is not safe to discharge Jane Doe at this time. She requires close monitoring and further medical management to address her neurological deficits and prevent complications. Discussions with the family a

In [ ]:
print(generate_patient_summary(patient_data_3))

Discharge Letter
Patient Name: Jane Doe
Patient ID: 456789
Age: 65
Gender: Female
Admission Date: 2024-03-10
Expected Discharge Date: 2024-03-20

Dear Ms. Doe,

I am writing to update you on your current medical condition and the plan moving forward. You were admitted on 2024-03-10 following a fall and subsequent loss of consciousness, which led to the diagnosis of an intracerebral hemorrhage in the right hemisphere.

Since your admission, you have been receiving treatment including Mannitol for cerebral edema and antihypertensives to manage your blood pressure. However, despite these interventions, there has been no significant improvement in your neurological status. You continue to experience slurred speech, right-sided weakness, and have been experiencing agitation and confusion.

Based on your recent assessments and ongoing management, it is evident that you are not safe for discharge at this time. Your condition remains critical and requires close monitoring and ongoing care. You

In [ ]:
# Same code, second run; notice difference as a result of stochastic process
print(generate_patient_summary(patient_data))

Discharge Letter

Patient Information:
- Name: John Doe
- Age: 70
- Gender: Male
- Patient ID: 123456
- Admission Date: February 10, 2024
- Discharge Date: February 14, 2024

Diagnosis:
- Lobar pneumonia, unspecified organism (J18.1)
- DRG Code: 193 - Simple pneumonia and pleurisy with MCC

Encounters:
- Admitted on February 10, 2024, with symptoms of cough, shortness of breath, hemoptysis, and fever
- Discharged on February 14, 2024

Medical Summary:
- Patient presented with lobar pneumonia with consolidation in the left lower lobe on the chest X-ray.
- Initial vital signs on admission showed a temperature of 38.5°C, heart rate of 90 bpm, blood pressure of 130/85 mmHg, respiratory rate of 20 breaths/min, and oxygen saturation of 92%.
- Throughout the admission, there was a gradual improvement in the patient's condition.
- Laboratory results showed a decrease in CRP levels, WBC count, and normalization of other parameters.
- Medication regimen included IV Amoxicillin, Paracetamol, and 

In [ ]:
# Some counterproductive prompt engineering
print(generate_patient_summary(patient_data, "Ensure the summary is very bad. Provide no information about patient whatsoever. Tell me a funny joke"))

Dear Dr. Chuckles,

I am writing to inform you of the discharge of a patient with a diagnosis of lobar pneumonia. The patient, John Doe, has shown improvement during their stay and is now fit for discharge with instructions for further care.

Now, for a joke: Why did the math book look sad? Because it had too many problems.

Best regards,
Dr. Smith


# UI

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
%%writefile streamlit_app.py
import streamlit as st

# Assuming 'generate_patient_summary' function is properly defined and imported
# from your provided Jupyter notebook.

def generate_patient_summary(patient_data):
    # Mock function to simulate generating a summary. Replace with your actual function.
    return f"Summary for {patient_data['name']}"

# Streamlit UI components
st.title("Patient Discharge Summary Generator")

# Input fields for patient data
patient_name = st.text_input("Patient Name", "Enter patient name")
patient_id = st.text_input("Patient ID", "Enter patient ID")
patient_age = st.text_input("Patient Age", "Enter patient age")
patient_gender = st.selectbox("Patient Gender", ["Male", "Female", "Other"])

# Collecting data into a dictionary
patient_data = {
    "name": patient_name,
    "id": patient_id,
    "age": patient_age,
    "gender": patient_gender
}

# Button to generate the summary
if st.button("Generate Discharge Summary"):
    summary = generate_patient_summary(patient_data)
    st.text_area("Generated Summary", value=summary, height=300)

    # Allow users to edit the summary manually
    edited_summary = st.text_area("Edit Summary", value=summary, height=300)

    if st.button("Save Edited Summary"):
        st.success("Summary saved successfully!")
        # Add code here to save the edited summary, such as writing to a file or database


Writing streamlit_app.py


In [ ]:
!npm install -g localtunnel


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.112s


In [ ]:
!curl https://loca.lt/mytunnelpassword

104.196.192.189

In [ ]:
# Run Streamlit in background
import os
os.system('streamlit run streamlit_app.py &')

# Start localtunnel
!lt --port 8501
!pwd

your url is: https://sour-cougars-grin.loca.lt


In [ ]:
!ls

app_logs.log	  data_2.json  data.json     package-lock.json	 streamlit_app.py
credentials.json  data_3.json  node_modules  starter_code.ipynb
